<a href="https://colab.research.google.com/github/n-vit/YaP_Projects/blob/main/7_%D0%9E%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5_%D1%81_%D1%83%D1%87%D0%B8%D1%82%D0%B5%D0%BB%D0%B5%D0%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из банка Б каждый месяц стали уходить клиенты. Немного, но заметно. Маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. У нас есть исторические данные о поведении клиентов и расторжении договоров с банком. 

Нужно построить модель с предельно большим значением *F1*-меры - не менее 0.59. 

Дополнительная метрика *AUC-ROC* 

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

**Признаки**

- RowNumber — индекс строки в данных
- CustomerId — уникальный идентификатор клиента
- Surname — фамилия
- CreditScore — кредитный рейтинг
- Geography — страна проживания
- Gender — пол
- Age — возраст
- Tenure — сколько лет человек является клиентом банка
- Balance — баланс на счёте
- NumOfProducts — количество продуктов банка, используемых клиентом
- HasCrCard — наличие кредитной карты
- IsActiveMember — активность клиента
- EstimatedSalary — предполагаемая зарплата

**Целевой признак**
- Exited — факт ухода клиента

## Подготовка данных

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.utils import shuffle

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 

from sklearn.metrics import accuracy_score 
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score

In [ ]:
data = pd.read_csv('/datasets/Churn.csv')
display(data.info())
display(data.head(3))
display(data.isna().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


None

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1


RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

В процессе подготовки:
- разберемся в пропусках в данных о Продолжительности сотрудничества
- проверим фрейм на дубликаты и удалим столбец с фамилиями (влияние фамилии на отток клиентов это не про математику пожалуй)
- категоризируем `obgect`, Страна проживания и Пол 
- понизим разрядность
- разберем фрейм на тренировочную, валидационную и тестовую выборки (60%/20%/20%)


In [ ]:
display(data[data['Tenure'].isna()].head(10))

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
30,31,15589475,Azikiwe,591,Spain,Female,39,NaN,0.00,3,1,0,140469.38,1
48,49,15766205,Yin,550,Germany,Male,38,NaN,103391.38,1,0,1,90878.13,0
51,52,15768193,Trevisani,585,Germany,Male,36,NaN,146050.97,2,0,0,86424.57,0
53,54,15702298,Parkhill,655,Germany,Male,41,NaN,125561.97,1,0,0,164040.94,1
60,61,15651280,Hunter,742,Germany,Male,35,NaN,136857.00,1,0,0,84509.57,0
82,83,15641732,Mills,543,France,Female,36,NaN,0.00,2,0,0,26019.59,0
85,86,15805254,Ndukaku,652,Spain,Female,75,NaN,0.00,2,1,1,114675.75,0
94,95,15676966,Capon,730,Spain,Male,42,NaN,0.00,2,0,1,85982.47,0
99,100,15633059,Fanucci,413,France,Male,34,NaN,0.00,2,0,0,6534.18,0
111,112,15665790,Rowntree,538,Germany,Male,39,NaN,108055.10,2,1,0,27231.26,0


В строках с пропущенной информацией о продолжительности сотрудничества встречаются разные записи по всем признакам, прямой зависимости между одним из признаков "Tenure" не видно. Пропуски составляют 9% фрейма. 

Удалим их из фрейма. 

Заодно сразу удалим и столбцы с информацией о фамилии/ ID пользователя и номере строки записи, поскольку они не влияют на поведение клиента, но могут снизить точность модели. 

In [ ]:
data = data.dropna()
data = data.drop(['Surname', 'RowNumber', 'CustomerId'], axis=1)

In [ ]:
display(data.duplicated().sum())

0

In [ ]:
#encoder = OrdinalEncoder()
#data = pd.DataFrame(encoder.fit_transform(data), columns=data.columns)

In [ ]:
print(data['Geography'].unique())

['France' 'Spain' 'Germany']


In [ ]:
data = pd.get_dummies(data, drop_first=True)

In [ ]:
df_float = data.select_dtypes(include=['float']) 
df_float = data.apply(pd.to_numeric,downcast='float')
data[df_float.columns] = df_float

df_int = data.select_dtypes(include=['int']) 
df_int = df_int.apply(pd.to_numeric,downcast='unsigned') 
data[df_int.columns] = df_int

In [ ]:
display(data.info())
display(data.head(3))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9091 entries, 0 to 9998
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CreditScore        9091 non-null   float32
 1   Age                9091 non-null   float32
 2   Tenure             9091 non-null   float32
 3   Balance            9091 non-null   float32
 4   NumOfProducts      9091 non-null   float32
 5   HasCrCard          9091 non-null   float32
 6   IsActiveMember     9091 non-null   float32
 7   EstimatedSalary    9091 non-null   float32
 8   Exited             9091 non-null   float32
 9   Geography_Germany  9091 non-null   uint8  
 10  Geography_Spain    9091 non-null   uint8  
 11  Gender_Male        9091 non-null   uint8  
dtypes: float32(9), uint8(3)
memory usage: 417.3 KB


None

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619.0,42.0,2.0,0.000000,1.0,1.0,1.0,101348.882812,1.0,0,0,0
1,608.0,41.0,1.0,83807.859375,1.0,0.0,1.0,112542.578125,0.0,0,1,0
2,502.0,42.0,8.0,159660.796875,3.0,1.0,0.0,113931.570312,1.0,0,0,0


Выглядит неплохо, продолжим

In [ ]:
target = data['Exited']
features = data.drop(['Exited'] , axis=1)
features_train, features_a, target_train, target_a = train_test_split(
    features, target, test_size=0.4, random_state=12345) 
features_valid, features_test, target_valid, target_test = train_test_split(
    features_a, target_a, test_size=0.5, random_state=12345) 
print('Размер обучающей выборки:', features_train.shape)
print('Размер валидационной выборки:', features_valid.shape)
print('Размер тестовой выборки:', features_test.shape)

Размер обучающей выборки: (5454, 11)
Размер валидационной выборки: (1818, 11)
Размер тестовой выборки: (1819, 11)


## Исследование задачи

In [ ]:
print('Доля оттока:', data.query('Exited == 1')['Exited'].count() / data['Exited'].count())

Доля оттока: 0.2039379606203938



    
Мы нашли дисбаланс класса. Проблема в том, что если мы будем обучать модель на сырых данных, без учета дисбаланса,  то она может переучиться на определение доминирующего класса и не уметь определять другой. Так же дисбаланс оказывает влияние на выбор метрики обучения(accuracy тут уже не подойдет).


20% записей клиентов со статусом "Выбыл"! действительно много. 

Выберем одну из моделей классификации для предсказаний оттока, обучив Решающее дерево, Случайный лес и Логистическую регрессию и сравнив базовые метрики:

**Решающее дерево**

In [ ]:
model_dtc = None
best_result = 0
best_depth = 0
for depth in range(1, 10):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if result > best_result:
        model_dtc = model
        best_result = result
        best_depth = depth
print('Accuracy наилучшей модели `DecisionTree` на валидационной выборке:', best_result)
print('Оптимальный параметр max_depth:', best_depth)
valid_predictions = model_dtc.predict(features_valid)
print()
print('Точность на валидационной выборке:', precision_score(target_valid, valid_predictions))
print('Полнота на валидационной выборке:', recall_score(target_valid, valid_predictions))
print('F1 на валидационной выборке:', f1_score(target_valid, valid_predictions))

probabilities_valid = model_dtc.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('Сравнение со случайной моделью AUC-ROC:', auc_roc)

Accuracy наилучшей модели `DecisionTree` на валидационной выборке: 0.8586358635863587
Оптимальный параметр max_depth: 6

Точность на валидационной выборке: 0.7884615384615384
Полнота на валидационной выборке: 0.4350132625994695
F1 на валидационной выборке: 0.5606837606837607
Сравнение со случайной моделью AUC-ROC: 0.8495399046859958


**Случайный лес**

In [ ]:
model_rfc = None
best_result = 0
best_est = 0
for est in range(1, 100):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(features_train, target_train) 
    result = model.score(features_valid, target_valid)
    if result > best_result:
        model_rfc = model
        best_result = result
        best_est = est

print("Accuracy наилучшей модели на валидационной выборке:", best_result)
print('Оптимальный параметр n_estimators:', best_est)
valid_predictions = model_rfc.predict(features_valid)
print()
print('Точность на валидационной выборке:', precision_score(target_valid, valid_predictions))
print('Полнота на валидационной выборке:', recall_score(target_valid, valid_predictions))
print('F1 на валидационной выборке:', f1_score(target_valid, valid_predictions))

probabilities_valid = model_rfc.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('Сравнение со случайной моделью AUC-ROC:', auc_roc)

Accuracy наилучшей модели на валидационной выборке: 0.8646864686468647
Оптимальный параметр n_estimators: 43

Точность на валидационной выборке: 0.7717842323651453
Полнота на валидационной выборке: 0.493368700265252
F1 на валидационной выборке: 0.6019417475728156
Сравнение со случайной моделью AUC-ROC: 0.853024443311361


**Логистическая регрессия**

In [ ]:
model_lr = LogisticRegression(random_state=12345, solver='liblinear') 
model_lr.fit(features_train, target_train)
valid_predictions = model_lr.predict(features_valid)
print('Accuracy модели `LogisticRegression` на валидационной выборке', accuracy_score(target_valid, valid_predictions))
print()
print('Точность на валидационной выборке:', precision_score(target_valid, valid_predictions))
print('Полнота на валидационной выборке:', recall_score(target_valid, valid_predictions))
print('F1 на валидационной выборке:', f1_score(target_valid, valid_predictions))

probabilities_valid = model_lr.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('Сравнение со случайной моделью AUC-ROC:', auc_roc)

Accuracy модели `LogisticRegression` на валидационной выборке 0.7887788778877888

Точность на валидационной выборке: 0.40540540540540543
Полнота на валидационной выборке: 0.03978779840848806
F1 на валидационной выборке: 0.07246376811594203
Сравнение со случайной моделью AUC-ROC: 0.6940674487397309


линейный Accurasy - 0,8 (если всегда предсказывать 0 - то в 80% случаев предсказание будет точным). Логистическая регрессия примерно так и делает, возможно из-за дисбаланса в таргете. У остальных моделей результаты интереснее. 

Лучшие результаты в чистой выборке показала модель **Случайный лес** с критерием n_estimators=98. До минимального приемлемого по условиям результата F1 меры нехватило всего одной сотой.  Попробуем улучшить эту модель, поработав с балансом таргета

## Борьба с дисбалансом

Сначала увеличим объем единиц в таргете (сейчас соотношение 1/5, значит можно повторить выборку с положительным таргетом пять раз)

In [ ]:
def upsample(features, target, repeat):
    features_zeros = features_train[target == 0]
    features_ones = features_train[target == 1]
    target_zeros = target_train[target == 0]
    target_ones = target_train[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled



In [ ]:
features_upsampled, target_upsampled = upsample(features_train, target_train, 5)

RandomForestClassifier(random_state=12345, n_estimators=98)
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)

print("F1 дополненной выборки:", f1_score(target_valid, predicted_valid))
print('Доля оттока в дополненной выборке:', target_upsampled[target == 1].count() / target_upsampled.count())


F1 дополненной выборки: 0.6160583941605839
Доля оттока в дополненной выборке: 0.5653745732074714


Выиграли пару сотых и получили приемлемый результат. Попробуем наоборот, уменьшить объем выборки с нулевыми значениями в пять раз:

In [ ]:
def downsample(features, target, fraction): #the function increases the data selection and balances the target
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

In [ ]:
features_downsampled, target_downsampled = downsample(features_train, target_train, 0.2)

RandomForestClassifier(random_state=12345, n_estimators=98)
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)

print("F1 сокращенной выборки:", f1_score(target_valid, predicted_valid))
print('Доля оттока в сокращенной выборке:', target_downsampled[target == 1].count() / target_downsampled.count())

F1 сокращенной выборки: 0.5699020480854853
Доля оттока в сокращенной выборке: 0.5652610441767069


Результаты downsample скромнее, а значит оставляем **модель Случайного леса с 98 деревьями на увеличенной выборке**. 

## Тестирование модели

Перед тестированием модели мы можем еще улучшить ее, обучив на увеличенной выборке (train + valid)

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.2, random_state=12345)
print('Размер обучающей выборки:', features_train.shape)
print('Размер тестовой выборки:', features_test.shape)

Размер обучающей выборки: (7272, 11)
Размер тестовой выборки: (1819, 11)


In [ ]:
features_upsampled, target_upsampled = upsample(features_train, target_train, 5)

RandomForestClassifier(random_state=12345, n_estimators=98)
model.fit(features_upsampled, target_upsampled)
test_predictions = model.predict(features_test)

print('Accuracy Случайного леса на тестовой выборке', accuracy_score(target_test, test_predictions))
print()
print('Точность на тестовой выборке:', precision_score(target_test, test_predictions))
print('Полнота на тестовой выборке:', recall_score(target_test, test_predictions))
print('F1 на тестовой выборке:', f1_score(target_test, test_predictions))

probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]

auc_roc = roc_auc_score(target_test, probabilities_one_test)

print('Сравнение со случайной моделью AUC-ROC:', auc_roc)

Accuracy Случайного леса на тестовой выборке 0.8521165475536009

Точность на тестовой выборке: 0.668918918918919
Полнота на тестовой выборке: 0.5365853658536586
F1 на тестовой выборке: 0.5954887218045114
Сравнение со случайной моделью AUC-ROC: 0.8513933277263808


Получилось! Целевой барьер значения F1 меры преодолен, модель на тестовой выборке показывает значения заметно лучше случайной выборки и линейной модели. 

Наша модель может с заданной точностью предсказывать отток клиентов по их поведению (набору фич), ее можно использовать для выявления потенциально уходящих клиентов и сокращения оттока. 



## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: задача исследована
    - [x]  Исследован баланс классов
    - [x]  Изучены модели без учёта дисбаланса
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 3: учтён дисбаланс
    - [x]  Применено несколько способов борьбы с дисбалансом
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 4: проведено тестирование
- [x]  Удалось достичь *F1*-меры не менее 0.59
- [x]  Исследована метрика *AUC-ROC*